In [1]:
# Check identifiability on longitudinal UKB age sex

import pickle

demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/UKB/'

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

print(demo.keys())

dict_keys(['f_22001_0_0', 'f_21003_3_0', 'f_21003_2_0', 'f_21000_0_0', 'CaAfCarDiff', 'CaIndDiff', 'CaChiDiff', 'f_21000_0_0_comb', 'longitudinal', '22438-0.0', '22828-0.0', '26201-0.0', '26202-0.0', '26203-0.0', '26204-0.0', '26205-0.0', '26206-0.0', '26207-0.0', '26208-0.0', '26209-0.0', '26210-0.0', '26211-0.0', '26212-0.0', '26213-0.0', '26214-0.0', '26215-0.0', '26216-0.0', '26217-0.0', '26218-0.0', '26219-0.0', '26220-0.0', '26221-0.0', '26222-0.0', '26223-0.0', '26224-0.0', '26225-0.0', '26226-0.0', '26227-0.0', '26228-0.0', '26229-0.0', '26231-0.0', '26232-0.0', '26233-0.0', '26234-0.0', '26235-0.0', '26236-0.0', '26237-0.0', '26238-0.0', '26239-0.0', '26240-0.0', '26241-0.0', '26242-0.0', '26243-0.0', '26244-0.0', '26245-0.0', '26246-0.0', '26247-0.0', '26248-0.0', '26249-0.0', '26250-0.0', '26251-0.0', '26252-0.0', '26253-0.0', '26254-0.0', '26255-0.0', '26256-0.0', '26257-0.0', '26258-0.0', '26259-0.0', '26260-0.0', '26261-0.0', '26262-0.0', '26263-0.0', '26264-0.0', '26265-

In [2]:
# Load longitudinal data and age

import numpy as np

scan2 = []
scan3 = []
sex = []
age = []

for sub in demo['longitudinal']:
    try:
        p2 = np.load(f'{demodir}/fc/{sub}_task-20227-2_fc.npy')
        p3 = np.load(f'{demodir}/fc/{sub}_task-20227-3_fc.npy')
        s = demo['f_22001_0_0'][sub]
        a = demo['f_21003_2_0'][sub]
    except KeyError:
        continue
    scan2.append(p2)
    scan3.append(p3)
    sex.append(s)
    age.append(a)
    
scan2 = np.stack(scan2)
scan3 = np.stack(scan3)
sex = np.array(sex)

print(len(scan2), len(scan3), len(sex), len(sex))

2658 2658 2658 2658


In [75]:
# Try simple model

from sklearn.linear_model import Ridge, LogisticRegression

def rmse(a, b):
    return np.mean((a-b)**2)**0.5

sz = 1300
errs = []

for n in range(0,2001,sz):
    xtr = scan2[n:n+sz]
    xt = scan3[n:n+sz]
    ytr = sex[n:n+sz]
    yt = sex[n:n+sz]

    reg = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    yhat = reg.predict(xt)
#     err = rmse(yhat, yt)
    err = np.mean(yhat == yt)
    print(err)
    errs.append(err)
    
print('---')
print(np.mean(errs), np.std(errs))

0.8838461538461538
0.8961538461538462
---
0.89 0.006153846153846176


In [1]:
import pickle

demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/Fibromyalgia/'

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

print(demo.keys())

dict_keys(['Fibromyalgia', 'Age', 'Pain_Score', 'Rheumatology_Score_1', 'Rheumatology_Score_2', 'Hamilton_Depression_Score', 'Hamilton_Anxiety_Score', 'Fibromyalgia_Impact_Score', 'Weight', 'Height', 'BMI'])


In [4]:
import numpy as np

rest = []
epr = []
fibro = []

for sub in demo['Fibromyalgia']:
    try:
        p0 = np.load(f'{demodir}/fc/{sub}_task-rest_fc.npy')
        p1 = np.load(f'{demodir}/fc/{sub}_task-epr_fc.npy')
        f = demo['Fibromyalgia'][sub]
        rest.append(p0)
        epr.append(p1)
        fibro.append(f == 'Yes')
    except:
        pass
    
rest = np.stack(rest)
epr = np.stack(epr)
fibro = np.array(fibro)

print(len(rest), len(epr), len(fibro))

65 65 65


In [9]:
# Logistic on Fibromyalgia

from sklearn.linear_model import LogisticRegression

def rmse(a, b):
    return np.mean((a-b)**2)**0.5

# Cheating

errs = []

for _ in range(10):
    idcs = np.random.permutation(65)[:50]

    xtr = rest[idcs]
    xt = epr[idcs]
    ytr = fibro[idcs]
    yt = fibro[idcs]

    reg = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    yhat = reg.predict(xt)
    err = np.mean(yhat == yt)
    print(err)
    errs.append(err)
    
print(np.mean(errs), np.std(errs))

0.88
0.9
0.86
0.86
0.88
0.78
0.84
0.82
0.86
0.88
0.8560000000000001 0.0332264954516723


In [16]:
# Not Cheating

from sklearn.model_selection import train_test_split

for _ in range(20):

    xrtr, xrt, xetr, xet, ytr, yt = train_test_split(rest, epr, fibro, stratify=fibro, train_size=50)

    xtr = xetr #np.concatenate([xrtr[:25], xetr[25:]])
    xt = xet #np.concatenate([xrt[:8], xet[8:]])

    reg = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    yhat = reg.predict(xt)
    err = np.mean(yhat == yt)
    print(err)
    errs.append(err)
    
print(np.mean(errs), np.std(errs))

0.5333333333333333
0.6
0.6
0.6
0.5333333333333333
0.7333333333333333
0.6
0.4
0.7333333333333333
0.5333333333333333
0.4666666666666667
0.6666666666666666
0.4
0.5333333333333333
0.6666666666666666
0.4
0.7333333333333333
0.26666666666666666
0.9333333333333333
0.5333333333333333
0.5733333333333334 0.15153065188778955


In [1]:
import pickle

demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

print(demo.keys())

dict_keys(['Race', 'Sex', 'Med_Rating', 'GAF001', 'GAF002', 'GAF003', 'age_at_cnb', 'battery_valid', 'PADT_GENUS', 'PADT_VALID', 'PADT_A', 'PADT_T', 'PADT_SAME_CR', 'PADT_SAME_RTCR', 'PADT_PC', 'PADT_SAME_PC', 'PFMT_GENUS', 'PFMT_VALID', 'PFMT_TP', 'PFMT_TN', 'PFMT_FP', 'PFMT_FN', 'PFMT_TPRT', 'PFMT_TNRT', 'PFMT_FPRT', 'PFMT_FNRT', 'PFMT_IFAC_TOT', 'PFMT_IFAC_RTC', 'PEIT_GENUS', 'PEIT_VALID', 'PEIT_CR', 'PEIT_CRT', 'PEITANG', 'PEITFEAR', 'PEITHAP', 'PEITNOE', 'PEITSAD', 'PEITANGRT', 'PEITFEARRT', 'PEITHAPRT', 'PEITNOERT', 'PEITSADRT', 'PWMT_GENUS', 'PWMT_VALID', 'PWMT_TP', 'PWMT_TN', 'PWMT_FP', 'PWMT_FN', 'PWMT_TPRT', 'PWMT_TNRT', 'PWMT_FPRT', 'PWMT_FNRT', 'PWMT_KIWRD_TOT', 'PWMT_KIWRD_RTC', 'PVRT_GENUS', 'PVRT_VALID', 'PVRT_CR', 'PVRT_RTCR', 'PVRT_RTER', 'PEDT_GENUS', 'PEDT_VALID', 'PEDT_HAP_CR', 'PEDT_SAD_CR', 'PEDT_ANG_CR', 'PEDT_FEAR_CR', 'PEDT_HAP_RTCR', 'PEDT_SAD_RTCR', 'PEDT_ANG_RTCR', 'PEDT_FEAR_RTCR', 'PEDT_SAME_CR', 'PEDT_SAME_RTCR', 'PEDT_A', 'PEDT_T', 'PEDT_PC', 'PEDT_HAP_P

In [31]:
import numpy as np

nback = []
emoid = []
wrat = []

for sub,r in demo['Race'].items():
    if r != 'AA':
        continue
    try:
        w = demo['WRAT_CR_STD'][sub]
        p0 = np.load(f'{demodir}/fc/{sub}_task-nback_fc.npy')
        p1 = np.load(f'{demodir}/fc/{sub}_task-emoid_fc.npy')
        wrat.append(w)
        nback.append(p0)
        emoid.append(p1)
    except:
        continue
        
nback = np.stack(nback)
emoid = np.stack(emoid)
wrat = np.array(wrat)

print(len(nback), len(emoid), len(wrat))

601 601 601


In [33]:
# Not Cheating

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

def rmse(a, b):
    return np.mean((a-b)**2)**0.5

errs = []
nullerrs = []

for i in range(20):

    xtr, _, xt, _, ytr, _ = train_test_split(emoid, nback, wrat, train_size=500)
    yt = ytr

    reg = Ridge(alpha=100).fit(xtr, ytr)
    yhat = reg.predict(xt)
    err = rmse(yhat, yt)
    nullerr = rmse(np.mean(ytr), yt)
    print(i, err, nullerr)
    errs.append(err)
    nullerrs.append(nullerr)
    
print('---')
print(np.mean(errs), np.std(errs))
print(np.mean(nullerrs), np.std(nullerrs))

0 11.451603030840223 14.70053209921328
1 11.472295202827562 14.760197695153002
2 11.81740062721352 14.611625508477829
3 11.430129122939736 14.89319361319123
4 11.94694450990803 14.828331126596815
5 11.819976724430354 14.801326832416073
6 11.352061112457193 14.439744596079253
7 11.447246115857903 14.318407173984122
8 10.911285741875016 14.421091636904608
9 11.823359795034575 14.662978005848608
10 11.539251431913 14.955488089661268
11 11.941842343521154 15.055550471503857
12 11.888884757594573 15.064496539878125
13 11.888562087269088 15.075868930181107
14 11.763205484275232 14.954684082253292
15 11.209559065563608 14.275038213609097
16 11.538132799657856 14.928744220462754
17 11.612897997320617 14.83007700586885
18 11.834741059127943 14.741491240712387
19 11.830553973832854 15.196602120210953
---
11.625996649173002 0.2699369415453031
14.775773460110326 0.2531826524455233
